In [542]:
import os
import sys
import shutil
sys.path.append('../Module')
folder_path = '../Dataset'

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, f1_score
from sklearn import set_config

from loader_csv import charger_csv
from data_to_csv import df_to_csv
from utils import generate_description
from currency import currency_cleaning
from naics_engineering import naicsEngineering

from Import_csv import import_csv
imported_csv_path = import_csv()

df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/65876671.py:31: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [543]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[3166973001, 1418265001, 3792654003, 909184400..."
1,Name,object,14,0.00,779583,"[KERMS SUPER FOODS INC, COLLIER POOL SERVICES ..."
2,City,object,30,0.00,32581,"[DENVER, HOUSTON, RUSHVILLE, ASHLAND, JESUP]"
3,State,object,14,0.00,51,"[CA, NH, WA, NY, LA]"
4,Zip,int64,0,0.00,33611,"[55369, 33127, 77964, 48141, 10001]"
5,Bank,object,1559,0.17,5802,"[STEARNS BK NATL ASSOC, ALTAPACIFIC BANK, TD B..."
6,BankState,object,1566,0.17,56,"[IL, GA, MS, TX]"
7,NAICS,int64,0,0.00,1312,"[448150, 236118, 812210, 441110, 443112]"
8,ApprovalDate,object,0,0.00,9859,"[15-Aug-05, 4-Feb-02, 6-Nov-06, 7-Dec-06, 11-O..."
9,ApprovalFY,object,0,0.00,70,"[2008, 2006, 2003, 2001, 1992]"


In [544]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [545]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)
df.NAICS.value_counts()

NAICS
44-45    127251
81        72618
54        68170
31-33     68029
72        67600
23        66646
62        55366
42        48743
56        32685
48-49     22531
71        14640
53        13632
51        11379
52         9496
11         9005
61         6425
21         1851
22          663
55          257
92          229
Name: count, dtype: int64

In [546]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]
df.NewExist.value_counts()

NewExist
1.0    502830
2.0    193520
Name: count, dtype: int64

In [547]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [548]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

df['RevLineCr'].value_counts()

RevLineCr
N    484187
Y    195147
Name: count, dtype: int64

In [549]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

df['LowDoc'].value_counts()

LowDoc
N    608823
Y     66203
Name: count, dtype: int64

In [550]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [551]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [552]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [553]:
df.shape

(675026, 13)

In [554]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[72, 62, 81, 48-49]"
1,Term,int64,0,0.0,377,"[120, 84, 55, 240]"
2,NoEmp,int64,0,0.0,499,"[2, 12, 7, 3]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 2]"
5,RetainedJob,int64,0,0.0,323,"[7, 5, 0, 2]"
6,UrbanRural,int64,0,0.0,3,"[1, 0]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,"[Y, N]"
9,GrAppv,int64,0,0.0,16880,"[250000, 833000, 5000, 68000, 25000]"


In [555]:
df_to_csv(df, folder_path, 'SBA_Cleaned_0.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_0.csv


In [556]:
df = pd.read_csv(imported_csv_path)

In [ ]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [ ]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [ ]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [ ]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [ ]:
valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [ ]:
valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [ ]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [ ]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [ ]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [ ]:
df.shape

(460816, 13)

In [ ]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[71, 62, 72, 54]"
1,Term,int64,0,0.0,373,"[84, 300, 102]"
2,NoEmp,int64,0,0.0,432,"[3, 4, 20, 18]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,192,"[0, 8]"
5,RetainedJob,int64,0,0.0,276,"[7, 0, 1]"
6,UrbanRural,int64,0,0.0,3,"[0, 1]"
7,RevLineCr,object,0,0.0,2,"[Y, N]"
8,LowDoc,object,0,0.0,2,[N]
9,GrAppv,int64,0,0.0,12824,"[20000, 55000, 70000, 25000, 102400]"


In [ ]:
df_to_csv(df, folder_path, 'SBA_Cleaned_1.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_1.csv


In [ ]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [ ]:
df.drop_duplicates(subset=['LoanNr_ChkDgt'], inplace=True)

In [ ]:
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [ ]:
df = df[(df.NewExist == 1.0) | (df.NewExist == 2.0)]

In [ ]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [ ]:
samples_bad_lol = df[(df['ChgOffDate'].notna())]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [ ]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & ((df['ChgOffPrinGr'] != 0.0) | (df['ChgOffPrinGr'] > 0.0))]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [ ]:
mapping = {'0': 'N', '1': 'Y'}
df['RevLineCr'] = df['RevLineCr'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['RevLineCr'].isin(valid_values)]

In [ ]:
mapping = {'0': 'N', '1': 'Y'}
df['LowDoc'] = df['LowDoc'].replace(mapping)

valid_values = ['N', 'Y']
df = df[df['LowDoc'].isin(valid_values)]

In [ ]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [ ]:
MIS_Status = pd.get_dummies(df['MIS_Status'], drop_first=True)
df.drop(['MIS_Status'], axis=1, inplace=True)
df = pd.concat([df, MIS_Status], axis=1)
df = df.rename(columns={'P I F': 'Approve'})

In [ ]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt', 'City', 'State', 'Bank', 'BankState', 'ApprovalDate', 'ApprovalFY']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df.dropna(inplace=True)

In [ ]:
df.shape

(675026, 13)

In [ ]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,NAICS,object,0,0.0,20,"[51, 54, 81, 44-45, 52]"
1,Term,int64,0,0.0,377,"[240, 60, 55, 84]"
2,NoEmp,int64,0,0.0,499,"[1, 2, 4, 25]"
3,NewExist,float64,0,0.0,2,"[1.0, 2.0]"
4,CreateJob,int64,0,0.0,216,"[0, 1]"
5,RetainedJob,int64,0,0.0,323,"[11, 15, 4, 0, 7]"
6,UrbanRural,int64,0,0.0,3,"[2, 1]"
7,RevLineCr,object,0,0.0,2,"[N, Y]"
8,LowDoc,object,0,0.0,2,"[N, Y]"
9,GrAppv,int64,0,0.0,16880,"[150000, 130000, 100000, 230000]"


In [ ]:
df_to_csv(df, folder_path, 'SBA_Cleaned_2.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_2.csv


In [ ]:
df = pd.read_csv(imported_csv_path)

/tmp/ipykernel_116678/3818106162.py:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(imported_csv_path)


In [ ]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,LoanNr_ChkDgt,int64,0,0.00,899164,"[7577974000, 9174523003, 7099464005, 510856500..."
1,Name,object,14,0.00,779583,"[GOLDEN SERVICES CORPORATION, MOBILE MENDERS, ..."
2,City,object,30,0.00,32581,"[FLORENCE, Chicago, ENFIELD, VADNAIS HEIGHTS, ..."
3,State,object,14,0.00,51,"[IN, CA, NJ, NY, FL]"
4,Zip,int64,0,0.00,33611,"[18964, 92843, 91355, 2886, 78702]"
5,Bank,object,1559,0.17,5802,"[AMERICAN BANK OF COMMERCE, BBCN BANK, BANK OF..."
6,BankState,object,1566,0.17,56,"[NC, MO, SD, MI, AL]"
7,NAICS,int64,0,0.00,1312,"[236115, 236118, 448140, 313221, 442210]"
8,ApprovalDate,object,0,0.00,9859,"[11-Jun-02, 29-Oct-91, 17-May-04, 27-Oct-99, 2..."
9,ApprovalFY,object,0,0.00,70,"[1996, 2007, 2004, 2010]"


In [ ]:
df = df.drop_duplicates(subset='LoanNr_ChkDgt')

In [ ]:
is_valide = {1.0, 2.0}
df['NewExist'] = df['NewExist'].isin(is_valide)

In [ ]:
is_valide = {'N','Y'}
df['RevLineCr'] = df['RevLineCr'].isin(is_valide)

In [ ]:
is_valide = {'N', 'Y'}
df['LowDoc'] = df['LowDoc'].isin(is_valide)

In [ ]:
is_valide = {'P I F', 'CHGOFF'}
df['MIS_Status'] = df['MIS_Status'].isin(is_valide)

In [ ]:
df = df.dropna(subset='NAICS')
df = df[df.NAICS != 0]
df['NAICS'] = df['NAICS'].apply(naicsEngineering)

In [ ]:
df['Franchise'] = np.where(df['FranchiseCode'] <= 1, 0, 1)
df = df.drop('FranchiseCode', axis=1)

In [ ]:
df['ApprovalFY'] = df['ApprovalFY'].astype(str).apply(lambda x: '20' + x[-2:] if int(x[-2:]) >= 0 and int(x[-2:]) <= 27 else '19' + x[-2:])
df['ApprovalFY'].unique()

In [ ]:
df['GrAppv'] = df['GrAppv'].apply(currency_cleaning).astype(float).astype(int)
df['SBA_Appv'] = df['SBA_Appv'].apply(currency_cleaning).astype(float).astype(int)
df['DisbursementGross'] = df['DisbursementGross'].apply(currency_cleaning).astype(float).astype(int)
df['BalanceGross'] = df['BalanceGross'].apply(currency_cleaning).astype(float).astype(int)
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).astype(float)

In [ ]:
samples_bad = df[(df['ChgOffPrinGr'].notna()) & (df['ChgOffPrinGr'] != 0.0)]
df.loc[samples_bad.index, 'MIS_Status'] = samples_bad['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(currency_cleaning).fillna(0.0).astype(float).astype(int)

In [ ]:
samples_bad_lol = df[df['ChgOffDate'].notna()]
df.loc[samples_bad_lol.index, 'MIS_Status'] = samples_bad_lol['MIS_Status'].replace(['P I F', np.nan], 'CHGOFF')

In [ ]:
df['Approve'] = df['MIS_Status'].apply(lambda x: 1 if x == 'P I F' else 0)
df.drop(['MIS_Status'], axis=1, inplace=True)

In [ ]:
leaking_feature = ['ChgOffPrinGr', 'BalanceGross', 'DisbursementGross', 'DisbursementDate', 'ChgOffDate']
useless_feature = ['Name', 'Zip', 'LoanNr_ChkDgt']
df = df.drop(leaking_feature, axis=1)
df = df.drop(useless_feature, axis=1)
df = df.dropna()

In [ ]:
generate_description(df)

,feature,data_type,null,nulPct,unique,uniqueSample
0,City,object,0,0.0,28860,"[TUCSON, BOISE, Washington, HENDERSON, PHILADE..."
1,State,object,0,0.0,51,"[NY, IL, MA, FL, CA]"
2,Bank,object,0,0.0,5221,"[HELENA NATIONAL BANK, CITIBANK, N.A., U.S. BA..."
3,BankState,object,0,0.0,55,"[AZ, NY, NC, TX, SD]"
4,NAICS,object,0,0.0,20,"[31-33, 72, 48-49, 71]"
5,ApprovalDate,object,0,0.0,7367,"[29-Jan-04, 21-Oct-04, 13-Jun-02, 20-Oct-93, 8..."
6,ApprovalFY,object,0,0.0,29,"[2004, 1993, 2003, 2006, 1999]"
7,Term,int64,0,0.0,378,"[240, 120, 300, 84]"
8,NoEmp,int64,0,0.0,502,"[0, 4, 2, 3]"
9,NewExist,bool,0,0.0,2,[True]


In [ ]:
df.shape

In [ ]:
df_to_csv(df, folder_path, 'SBA_Cleaned_Pure.csv')

Le DataFrame a été converti avec succès en fichier CSV : ../Dataset/SBA_Cleaned_Pure.csv
